In [1]:
from GreyNsights.analyst import DataWorker, DataSource, Pointer, Command, Analyst
from GreyNsights.frameworks import framework
import numpy as np

In [2]:
frameworks = framework()

pandas = frameworks.pandas

identity = Analyst("Alice", port=65441, host="127.0.0.1")

worker = DataWorker(port=65441, host="127.0.0.1")
dataset = DataSource(identity, worker, "Sample Data")

a = dataset.get_config()
print(a)

a = a.approve().init_pointer()



owner_name: Bob
dataset_name: Sample Data
privacy_budget: 10.0
trusted-aggregator: None
secret-sharing: Shamirs_scheme
private_columns: 

	-N
	-o
	-n
	-e
visible_columns: 
restricted_columns: default
allowed_queries: default
visible_queries: 
	-sum
	-count
	-mean
	-percentile
	-max
	-min
	-median
restricted_queries: None




In [3]:
df = pandas.DataFrame(a)

# 31.019322070944433
# 31.083416873168627
# 2850

# 31.011228070175438
# 30.0
# 2850


p = 3

p = dataset.send(p).chain

# print(df["TMC"].sum())

print(df["TMC"])

# top 5 rows
# print(df["TMC"].sum().get())
# top 50 rows


# last 5 rows
print(df.tail())
# last 50 rows
df.tail(50)

print(df.describe().get())

print("TMC sum: ", df["TMC"].sum().get())
print("TMC std: ", df["TMC"].std().get())
print("Severity mean: ", df["Severity"].mean().get())
print("Severity mean: ", df["Severity"].mean().get())

# print(df["TMC"])

# print(df["TMC"].mean())
# no of rows in dataframe
# print(len(df))
# Shape of Dataframe
df.shape

print("COLUMNS: ", df.columns)
print("SHAPE: ", df.shape)

df.columns = [
    "ID",
    "Source",
    "TMC",
    "Severity",
    "Start_Time",
    "End_Time",
    "Start_Lat",
    "Start_Lng",
    "End_Lat",
    "End_Lng",
    "Distance_mi",
    "Description",
    "Number",
    "Street",
    "Side",
    "City",
    "County",
    "State",
    "Zipcode",
    "Country",
    "Timezone",
    "Airport_Code",
    "Weather_Timestamp",
    "Temperature_F",
    "Wind_Chill_F",
    "Humidity_%",
    "Pressure_in",
    "Visibility_mi",
    "Wind_Direction",
    "Wind_Speed_mph",
    "Precipitation_in",
    "Weather_Condition",
    "Amenity",
    "Bump",
    "Crossing",
    "Give_Way",
    "Junction",
    "No_Exit",
    "Railway",
    "Roundabout",
    "Station",
    "Stop",
    "Traffic_Calming",
    "Traffic_Signal",
    "Turning_Loop",
    "Sunrise_Sunset",
    "Civil_Twilight",
    "Nautical_Twilight",
    "Astronomical_Twilight",
]

df = df[
    [
        "ID",
        "Source",
        "TMC",
        "Severity",
        "Start_Time",
        "End_Time",
        "Start_Lat",
        "Start_Lng",
        "End_Lat",
        "End_Lng",
    ]
]


df.dtypes


df["Somecol"] = (df["TMC"] + df["Severity"] / 10) / 2
(df["TMC"] + df["Severity"])

df["LOL"] = df["TMC"]


df["Somecol"] = df["TMC"] + df["Severity"]

(df["TMC"] + df["Severity"] / 10) / 2

df["TMC"] > 2

(df["Severity"] > 8) | (df["TMC"] > 200)

df[df["TMC"] > 200]

df[(df["Severity"] > 8) | (df["TMC"] > 200)]

And_df = df[(df["TMC"] > 200)]
# Multiple conditions: OR
Or_df = df[(df["Severity"] > 8) | (df["TMC"] > 200)]

df["Somecol"] = df["TMC"]


def somefunc(x):
    return x + 2


somefunc_pt = dataset.send(somefunc)


df["Somecol"] = df["TMC"].apply(somefunc_pt)


from math import cos, sin, asin, sqrt, pi


def haversine_distance(lat1, lon1, lat2, lon2):
    # Haversine distance formula taken from Michael Dunn's StackOverflow post:
    # https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

    x_1 = pi / 180 * lat1
    y_1 = pi / 180 * lon1
    x_2 = pi / 180 * lat2
    y_2 = pi / 180 * lon2

    dlon = y_2 - y_1
    dlat = x_2 - x_1
    a = sin(dlat / 2) ** 2 + cos(x_1) * cos(x_2) * sin(dlon / 2) ** 2

    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers

    return c * r


haversine_distance_pt = dataset.send(haversine_distance)


def lol1(x):

    return haversine_distance_pt(
        x["Start_Lat"], x["Start_Lng"], x["End_Lat"], x["End_Lng"]
    )


lol_pt = dataset.send(lol1)

df["hDistance"] = df.apply(
    lol_pt,
    axis=1,
)


# Single condition
df_dis_gt_2 = df[df["hDistance"] > 2]
# Multiple conditions: AND
And_df = df[(df["hDistance"] > 8) & (df["TMC"] > 200)]
# Multiple conditions: OR
Or_df = df[(df["hDistance"] > 8) | (df["TMC"] > 200)]
# Multiple conditions: NOT
Not_df = df[~((df["hDistance"] > 8) | (df["TMC"] > 200))]


Pointer->Bob
	 	 dataset:Sample Data
	 	 dtype:<class 'pandas.core.series.Series'>
	 	 id:203976078570
	 	 port:65441
	 	 host:127.0.0.1


Pointer->Sample Data
	 	 dataset:Sample Data
	 	 dtype:<class 'pandas.core.frame.DataFrame'>
	 	 id:525522738968
	 	 port:65441
	 	 host:127.0.0.1

                TMC      Severity     Start_Lat     Start_Lng       End_Lat  \
count  2.478818e+06  3.513617e+06  3.513617e+06  3.513617e+06  1.034799e+06   
mean   2.080226e+02  2.339929e+00  3.654195e+01 -9.579151e+01  3.755758e+01   
std    2.076627e+01  5.521935e-01  4.883520e+00  1.736877e+01  4.861215e+00   
min    2.000000e+02  1.000000e+00  2.455527e+01 -1.246238e+02  2.457011e+01   
25%    2.010000e+02  2.000000e+00  3.363784e+01 -1.174418e+02  3.399477e+01   
50%    2.010000e+02  2.000000e+00  3.591687e+01 -9.102601e+01  3.779736e+01   
75%    2.010000e+02  3.000000e+00  4.032217e+01 -8.093299e+01  4.105139e+01   
max    4.060000e+02  4.000000e+00  4.900220e+01 -6.711317e+01  4.907500e+01   



EOFError: Ran out of input